### Лекция 2. Функции, пространства имён, заголовочные файлы, cmake, юнит-тестирование

##### Функции, объявление и определение (declaration / definition)

Определение (definition) функции - описание её "интерфейса" (сигнатуры, возвращаемого типа и квалификаторов) И реализации.

```c++
float abs(float x)
{
    if (x >= 0)
        return x;
    return -x;
}

float min_value(const std::vector<float>& items)
{
    float rv = items.front();
    for (float x : items)
        rv = std::min(rv, x);
    return rv;
}
```

Замечание:
* для поиска минимума, конечно же, используйте `std::min_element`
* а для вычисления абсолютных величин функцию `abs` из `#include <cmath>`

Объявление (declaration) функции - описание её "интерфейса"

```c++
float abs(float x);

float min_value(const std::vector<float>& items);
```

Если не прописано явно, у функции в программе может быть несколько объявлений, но не больше одного определения.

__Вопрос__: что помещается в header-файл, что в cpp-файл?

<br />

##### Передача аргументов в функцию

Передача по значению - создание копии аргумента

```c++
float min_value(std::vector<float> x);
```

Передача по ссылке - работа с аргументом

```c++
float min_value(std::vector<float>& x);
```

Передача по const-ссылке - работа с аргументом и запрет на модификацию

```c++
float min_value(const std::vector<float>& x);
```

**Упражнение:** что здесь происходит с аргументами?

```c++
float sqr(const float x);

float min_value(std::vector<float>* x);
float min_value(const std::vector<float>* x);
float min_value(std::vector<float>* const x);
```

**Вопрос:** Когда лучше передавать по значению, а когда - по ссылке?

<br />

##### Возвращаемое значение

Функция ничего не возвращает:

```c++
void say_hello(const std::string& name)
{
    std::cout << "hello, " << name;
}
```

Возврат результата через возвращаемое значение (предпочтительный вариант):

```c++
std::vector<std::string> make_team()
{
    return { "Bifur", "Bofur", "Bombur", "Oin",
             "Gloin", "Fili", "Nori", "Dori",
             "Dwalin", "Ori", "Balin", "Kili" };
}
```

Возврат результата через аргумент (менее предпочтительный вариант в силу меньшей читаемости):

```c++
bool append_teamlead(Point3D location, std::vector<std::string>& team)
{
    if (is_inside(location, get_village("Shire")))
    {
        team.push_back("Thorin");
        return true;
    }
    return false;
}
```

Сравните:
    
```c++
// легко читается, что есть результат функции
std::vector<std::string> team = make_team();

// не очевидно, что результатом функции является
// изменение второго аргумента, нужно лезть в
// документацию или реализацию, чтобы понять
// замысел автора
append_teamlead(get_current_location(), team);

```

<br />

##### Ошибки при работе с аргументами и возвращаемыми значениями

```c++
const std::string* find_dwarf(const std::vector<std::string>& team, const std::string& name)
{
    for (const std::string &dwarf : team)
        if (dwarf == name)
            return &dwarf;
    return nullptr;
}

// usage 1
std::vector<std::string> team = make_team();
if (const std::string* ptr = find_dwarf(team, "Kuzya"))
    std::cout << *ptr;

// usage 2
if (const std::string* ptr = find_dwarf(make_team(), "Balin"))
    std::cout << *ptr;  // ???

// usage 3
if (const std::string* ptr = find_dwarf({"Ori", "Gloin", "Balin"}, "Balin"))
    std::cout << *ptr;  // ???
```

__Вопрос__: что будет напечатано программой ниже?

Показать пример на godbolt.org на clang 8.0.0 с разными оптимизациями, чтобы наглядно продемонстрировать ub

```c++
#include <iostream>
#include <string>

const std::string& f(const bool x,
                     const std::string& s1,
                     const std::string& s2)
{
    return x ? s1 : s2;
}

int main()
{
    const std::string& s = f(true, "123", "12345");
    std::cout << s << endl;
    return 0;
}
```

<br />

##### Значения аргументов по умолчанию

Можно задавать значения аргументов по умолчанию:

```c++
std::string convert_to_string(int value, int base = 10);
```

```c++
std::string join_strings(const std::vector<std::string>& strings,
                         const std::string& sep = "\n");
```

использование:

```c++
std::string s1 = convert_to_string(42);    // 42
std::string s2 = convert_to_string(42, 2); // 101010

std::string song = join_strings({"In the town where I was born",
                                 "Lived a man who sailed to sea",
                                 "And he told us of his life",
                                 "In the land of submarines",
                                });
                                 
std::string sentence = join_strings({"Nobody",
                                     "expects",
                                     "the",
                                     "spanish",
                                     "inquisition",
                                    },
                                    " ");
```

Но такие аргументы должны быть последними:

```c++
std::string join_strings(const std::vector<std::string>& strings,
                         const std::string& sep = "\n",
                         bool skip_empty_lines);  // compilation ERROR!
```

```c++
std::string join_strings(const std::vector<std::string>& strings,
                         const std::string& sep = "\n",
                         bool skip_empty_lines = false);  // OK
```

<br />

##### Перегрузка функции

https://en.cppreference.com/book/intro/function_overloading

Задача - реализовать конвертацию всего в строку. Желательно единообразно и чтобы: есть способ - компилируется и работает, нет способа - не компилируется.

```c++
std::string convert_to_string(int x);       // 1
std::string convert_to_string(unsigned x);  // 2
std::string convert_to_string(float x);     // 3

std::cout << convert_to_string(5);    // 1
std::cout << convert_to_string(5u);   // 2
std::cout << convert_to_string(5.f);  // 3
```

Для такого набора функций `convert_to_string` компилятор (clang 10.0) сгенерирует символы:
* `_Z17convert_to_stringB5cxx11i`
* `_Z17convert_to_stringB5cxx11j`
* `_Z17convert_to_stringB5cxx11f`

Т.е. тип аргумента - часть имени символа (для функции) при компиляции.

<br />

##### Пространства имён и name mangling при компиляции

https://en.wikipedia.org/wiki/Name_mangling

Пространства времён решают проблему, когда функция с одинаковым именем имеет две реализации в разных библиотеках.

Рассмотрим сценарий, где вы с другом / подругой пишете свой личный Half Life 3, вы работаете над ИИ соперников, а компаньон - над анимациями объектов. И вам, и компаньону для отладки нужно вычленять из сцены интересующие объекты, каждому нужна функция:

```c++
// EnemyAI.h
// Получить объекты сцены, с которыми может взаимодействовать соперник (укрытия, маркеры огневых точек, цели ...)
std::vector<Object*> collect_objects(const Scene& scene);

// Animations.h
// Получить анимируемые объекты сцены
std::vector<Object*> collect_objects(const Scene& scene);
```

Такой код некорректен, т.к. для функции `collect_objects` нарушено ODR-правило (One Definition Rule).

Вы можете поспорить с тем, что имена для функций из примера выбраны неудачно, но давайте остановимся на том факте, что иногда при выборе имён программисты между собой пересекаются.

```c++
// EnemyAI.h
namespace EnemyAI
{
  // Получить объекты сцены, с которыми может взаимодействовать соперник (укрытия, маркеры огневых точек, цели ...)
  std::vector<Object*> collect_objects(const Scene& scene);
}

// Animations.h
namespace Animations
{
  // Получить анимируемые объекты сцены
  std::vector<Object*> collect_objects(const Scene& scene);
}
```

Такой код слинкуется корректно, т.к. работает механизм name mangling во время компиляции.

Компилятор (clang 10.0) сгенерирует такие имена символов для функций:

* `_ZN7EnemyAI15collect_objectsERK5Scene`
* `_ZN10Animations15collect_objectsERK5Scene`

<br />

##### Поиск функции и `using namespace`

Точные правила поиска:

https://en.cppreference.com/w/cpp/language/lookup

При компиляции куска кода:

```c++
namespace json_parser {
namespace input_processing {

int read_int(const std::string& s)
{
    if (avx_instructions_available())
        return read_int_avx(s);
    else
        return read_int_default(s);
}
    
}
}
```

Компилятору нужно найти кандидатов для вызова `avx_instructions_available`, `read_int_avx` и `read_int_default`. Компилятор осуществляет их поиск в пространствах имён:
* глобальное
* `json_parser`
* `json_parser::input_processing`
* пространства имён аргументов функций (ничего для `avx_instructions_available`, `std` для `read_int_avx` и `read_int_default`)

Если для какой-нибудь из функций находится либо более одного кандидата для вызова либо ни одного - ошибка компиляции. Кандидат должен быть и быть только один.

<br />

**Замечание:** `using namespace X` - заполнить текущее пространство имён до закрывающей скобки именами из пространства X.

**Пример 1:**

Вызывающая программа:

```c++
#include "json_parser.h"

int main()
{
    std::string s = "123";

    // не скомпилируется, т.к. в глобальном пространстве имён нет функции read_int
    std::cout << read_int(s);
    
    // скомпилируется
    std::cout << json_parser::input_processing::read_int(s);
    
    // скомпилируется, т.к. глобальное пространство имён расширено
    using namespace json_parser::input_processing;
    std::cout << read_int(s);
    
    return 0;    
}
```

**Пример 2:**

```c++
int main() {
    std::cout << 1 << std::string("23");
    return 0;
}
```

или

```c++
using namespace std; 
// имена из std доступны в глобальном пространстве имён до конца файла

int main() {
    cout << 1 << string("23");
    return 0;
}
```

или

```c++
int main()
{
    using namespace std; 
    // имена из std доступны в глобальном пространстве имён до конца main

    cout << 1 << string("23");
    return 0;
}
```

<br />

**Рекомендации:**

1. Никогда не пишите `using namespace` в `.h` - файлах (почему?)
2. Ограничивайте область действия `using namespace` рационально.

**Пример рациональности:** функция, считающая время выполнения другой функции (многословность `std::chrono`):

```c++
// без using namespace
std::uint64_t get_execution_time_microseconds(const std::function<void()>& f)
{
    const std::chrono::high_resolution_clock::time_point start_ts =
        std::chrono::high_resolution_clock::now();
    f();    
    const std::chrono::high_resolution_clock::time_point final_ts =
        std::chrono::high_resolution_clock::now();    
    return std::chrono::duration_cast<std::chrono::microseconds>(final_ts - start_ts).count();
}

// с using namespace
std::uint64_t get_execution_time_microseconds(const std::function<void()>& f)
{
    using namespace std::chrono;    
    const high_resolution_clock::time_point start_ts = high_resolution_clock::now();
    f();    
    const high_resolution_clock::time_point final_ts = high_resolution_clock::now();    
    return duration_cast<microseconds>(final_ts - start_ts).count();
}

// с using namespace + auto
std::uint64_t get_execution_time_microseconds(const std::function<void()>& f)
{
    using namespace std::chrono;    
    const auto start_ts = high_resolution_clock::now();
    f();    
    const auto final_ts = high_resolution_clock::now();    
    return duration_cast<microseconds>(final_ts - start_ts).count();
}
```

<br />

##### Чтение из файла

Чтение из файла в стиле С

```c++
#include <cstdio>

int main()
{
	FILE* f = std::fopen("input.txt", "r");
	if (!f)
	{
		std::puts("cannot open file input.txt");
		return 1;
	}

	int sum = 0;
	int x = 0;
	while (std::fscanf(f, "%i", &x) != EOF)
		sum += x;

	std::printf("sum = %i\n", sum);

	std::fclose(f);

	return 0;
}
```

Чтение из файла в стиле C++

```c++
#include <fstream>
#include <iostream>


int main()
{
	std::ifstream ifs("input.txt");
	if (!ifs)
	{
		std::cerr << "ERROR: cannot open file input.txt" << std::endl;
		return 1;
	}

	int sum = 0, x = 0;
	while (ifs >> x)
		sum += x;

	std::cout << sum << std::endl;

	return 0;
}
```

Чтение из файла блоками

```c++
#include <fstream>
#include <iostream>
#include <vector>


int main()
{
	std::ifstream ifs("input.txt");
	if (!ifs)
	{
		std::cerr << "ERROR: cannot open file\n";
		return 1;
	}

	const unsigned BUFSIZE = 4096; 
	std::vector<char> buffer(BUFSIZE);

	while (true)
	{
		ifs.read(&buffer[0], buffer.size());
		const unsigned n = ifs.gcount();
		if (!n)
			break;

		std::string s(&buffer[0], n);
		std::cout << s;
	}

	return 0;
}
```

<br />

##### istream / ostream

Возможно, вы заметили, в С++ работа с чтением из файла через механизм потоков очень похожа с вводом-выводом из консоли.

Это не случайно. Класс, который отвечает за чтение из потока: `std::istream`.

Чтение из потока выглядит так:

```c++
int x;
std::cin >> x;
```

Класс `std::ifstream` наследуется от него. Файл представляется как поток данных. Чтение из файла:

```c++
std::ifstream ifs("input.txt");
int x;
ifs >> x;
```

Механизм потоков даёт возможность программисту дёшево абстрагироваться от источника данных (файл, поток ввода, память ...) и написать код, который будет работать с произвольным источником.

**Пример:** Напишем функцию, которая читает `int` из потока:

```c++
int read_int(std::istream& is)
{
    int x;
    is >> x;
    return x;
}
```

Вызывать эту функцию мы можем для любого потока:

```c++
int value1 = read_int(std::cin);  // чтение целого из потока stdin

std::ifstream ifs("input.txt");
int value2 = read_int(ifs);       // чтение целого из файла

std::stringstream ss("36");
int value3 = read_int(ss);        // чтение целого из памяти, представленной std::stringstream
```

<br />

##### Юнит-тесты

Контракты функции:
  * предусловие (precondition / expect)
  * постусловие (postcondition / ensure)
  * инвариант (precondition + postcondition / invariant)


Приведите примеры контрактов (exception-ы исключаем из рассмотрения, "сейчас про них ничего не знаем"):

```c++
double sqrt(double x);
bool binary_search(int *arr, int n, int value);
unsigned read_n(std::istream& is);
bool parse_int(const std::string &s, int &result);
```

Типы функциональных тестов:
* позитивный сценарий
* негативный сценарий
* граничные условия

Приведите примеры тестов на функции `sqrt`, `binary_search`, `read_n`, `parse_int`

Для упражнения напишем тесты на функцию, считающую длину ломаной

```c++
// polyline.h
#pragma once
#include <vector>

struct Point
{
	float x;
	float y;
};

float get_polyline_len(const std::vector<Point>& polyline);
```

```c++
// polyline.cpp
#include "polyline.h"
#include <cmath>

float get_polyline_len(const std::vector<Point>& polyline)
{
    float rv = 0;
    for (int i = 1; i < polyline.size(); ++i)
    {
        const Point prev = polyline[i - 1];
        const Point curr = polyline[i];
        const float dx = curr.x - prev.x;
        const float dy = curr.y - prev.y;
        rv += std::sqrt(dx * dx + dy * dy);
    }
    return rv;
}
```

```c++
// polyline_test.cpp
#include "polyline.h"
#include "gtest/gtest.h"

TEST(get_polyline_len, empty_polyline)
{
	std::vector<Point> empty_poly;
	const double len = get_polyline_len(empty_poly);
	EXPECT_EQ(0, len);
}

TEST(get_polyline_len, single_point_polyline)
{
	std::vector<Point> poly{{1,1}};
	const double len = get_polyline_len(poly);
	EXPECT_EQ(0, len);
}

// ???
```

<br />

##### Назначение header-файлов и cpp-файлов

Файлы в С++-программах делятся на 2 типа: компилируемые и включаемые:
    
* _Компилируемые_ файлы (`file.cpp`) - файлы с текстом на языке С++, подаются на вход компилятору для генерации соответствующего объектного файла с машинными интсрукциями (`file.o`)
* _Включаемые_ файлы (`file.h`) - файлы с текстом на языке С++, не являются конечной целью компиляции, их содержимое копируется в `cpp`-файлы при их компиляции через директиву `#include`

**Пример:** 

Напишем программу, по целому числу `n` считающую $(1 + 1/n)^n$.

Программа будет состоять из трёх файлов:
* `exp.cpp` - определение (definition) функции расчёта
* `exp.h` - декларация функции расчёта
* `main.cpp` - определение (definition) `main`
* `test_exp.cpp` - тесты

_Подробно объяснить программу_

Файл `exp.cpp`:

```c++
#include "exp.h"
#include <cmath>

double approximate_exp(const unsigned int n)
{
    if (n == 0)
        return 1.0;
    
    return std::pow(1.0 + 1.0 / n, n);
}
```

Файл `exp.h`:

```c++
#pragma once

double approximate_exp(const unsigned int n);
```

Файл `main.cpp`:

```c++
#include "exp.h" // <---
#include <iostream>

int main()
{
    unsigned n = 0;
    std::cin >> n;
    
    std::cout << approximate_exp(n);    
    return 0;
}
```

Файл `test_exp.cpp`:

```c++
#include "exp.h"  // <---
#include <gtest/gtest.h>

TEST(approximate_exp, zero)
{
    EXPECT_FLOAT_NEAR(approximate_exp(0), 1.0, 1e-3);
}
```

<br />

##### CMake-обвязка над C++ - проектами

Проштудировать содержимое `lab1_stub`, не забыть рассказать про схему компиляции.

Скрипт сборки:

In [7]:
!rm -rf build && mkdir build && cd build && cmake ../../lab1_stub/hasher && cmake --build .

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done
-- Generating done
-- Build files have been written to: /home/ivafanas/Documents/cpp_shad_students/2021/sem1/lecture_2_functions_namespace_cmake_unit_tests/build/googletest-download
Scanning dependencies of target googletest
[ 11%] Creating directories for 'googletest'
[ 22%] Performing download step (git clone) for 'googletest'
Cloning into 'googletest-src'...
Already on 'master'
Your branch is up to date with 'origin/master'.
[ 33%] Performing update step for 'googletest

Содержимое папки build:

In [8]:
!ls build

bin		cmake_install.cmake  googletest-src  lib
CMakeCache.txt	googletest-build     hash	     libhashlib.a
CMakeFiles	googletest-download  hash_unittests  Makefile


Прогоним тесты:

In [9]:
!build/hash_unittests

[==========] Running 3 tests from 1 test suite.
[----------] Global test environment set-up.
[----------] 3 tests from function1
[ RUN      ] function1.scenario1
[       OK ] function1.scenario1 (0 ms)
[ RUN      ] function1.scenario2
[       OK ] function1.scenario2 (0 ms)
[ RUN      ] function1.scenario3
/home/ivafanas/Documents/cpp_shad_students/2021/sem1/lab1_stub/hasher/src/tests/hash_unittest.cpp:17: Failure
Expected equality of these values:
  1
  2
[  FAILED  ] function1.scenario3 (0 ms)
[----------] 3 tests from function1 (0 ms total)

[----------] Global test environment tear-down
[==========] 3 tests from 1 test suite ran. (0 ms total)
[  PASSED  ] 2 tests.
[  FAILED  ] 1 test, listed below:
[  FAILED  ] function1.scenario3

 1 FAILED TEST


<br />

**Выдать первое домашнее задание если ещё не**